# prepare data 

In [310]:
import pandas as pd
from datetime import timedelta

### Пример данных

In [311]:
# Примерные входные данные (замените своими)
# df_investments: date, amount
df_investments = pd.DataFrame({
    'date': pd.date_range('2020-01-01', periods=36, freq='MS'),
    'amount': [150]*36
})

# df_rates: date, rate (где rate - годовая ставка, например 0.06 для 6%)
df_rates = pd.DataFrame({
    'date': pd.date_range('2020-01-01', periods=36, freq='MS'),
    'rate': [0.10, 0.10, 0.10] * 12
    #'rate': [0.06, 0.05, 0.07] * 12
})

In [312]:
df_investments

,date,amount
0,2020-01-01,150
1,2020-02-01,150
2,2020-03-01,150
3,2020-04-01,150
4,2020-05-01,150
5,2020-06-01,150
6,2020-07-01,150
7,2020-08-01,150
8,2020-09-01,150
9,2020-10-01,150


### Чтение данных 

In [313]:
df_investments.to_excel('investments.xlsx')

df_rates = pd.read_excel("Процентная ставка вклады 2010-2024.xlsx")
df_rates = df_rates.rename(columns={'Декада':'date', "Ставка, %":"rate"})

df_investments = pd.read_excel("Вклады.xlsx")


df_rates['rate'] = df_rates['rate'] * 0.01

### Глобальные настройки.

In [314]:
# Параметры
n = 450  # минимальная сумма для открытия вклада
investment_duration = 12  # продолжительность вклада в месяцах

# Объединяем данные по датам
df = pd.merge(df_investments, df_rates, on='date', how='left')

In [315]:
if df.isna().sum().sum() != 0:
    raise ValueError("Есть пропуск. Проверить данные.")

# Обработка вкладов, первый собранные вклады.

In [316]:
# Рассчитаем доходность по вкладам
def calculate_income(df, min_sum, duration):
    total_income = []
    balance = 0  # текущий баланс для инвестирования
    start_dates = []

    for i, row in df.iterrows():
        balance += row['amount']
        if balance >= min_sum:
            # Открываем вклад
            start_date = row['date']
            end_date = start_date + pd.DateOffset(months=duration)
            deposit_rate = df[(df['date'] == start_date)]['rate'].mean()
            # deposit_rate = df[(df['date'] >= start_date) & (df['date'] < end_date)]['rate'].mean()
            print("deposit_rate", deposit_rate)
            
            if pd.notna(deposit_rate):
                income = balance * deposit_rate * (duration / 12)
                total_income.append({
                    'start_date': start_date,
                    'end_date': end_date,
                    'principal': balance,
                    'rate': deposit_rate,
                    'income': income
                })
            
            # Обнуляем баланс после открытия вклада
            balance = 0
    
    return pd.DataFrame(total_income)

# Вычисление доходности первичных вкладов
df_income = calculate_income(df, n, investment_duration)
print(df_income)


deposit_rate 0.09514
deposit_rate 0.09561
deposit_rate 0.09521000000000002
deposit_rate 0.09538
deposit_rate 0.0977
deposit_rate 0.09736
deposit_rate 0.09936
deposit_rate 0.10625
deposit_rate 0.10534
deposit_rate 0.094
deposit_rate 0.09460000000000002
deposit_rate 0.0965
deposit_rate 0.0965
deposit_rate 0.09835
deposit_rate 0.09975
deposit_rate 0.0994
deposit_rate 0.0969
deposit_rate 0.09445
deposit_rate 0.0926
deposit_rate 0.09050000000000001
deposit_rate 0.0889
deposit_rate 0.08664999999999999
deposit_rate 0.084
deposit_rate 0.08449999999999999
deposit_rate 0.08310000000000001
deposit_rate 0.0827
deposit_rate 0.08345000000000001
deposit_rate 0.08615
deposit_rate 0.08635
deposit_rate 0.08835000000000001
deposit_rate 0.09005
deposit_rate 0.09145
deposit_rate 0.09325
deposit_rate 0.09502000000000001
deposit_rate 0.09895
deposit_rate 0.1058
deposit_rate 0.15325
deposit_rate 0.14705000000000001
deposit_rate 0.1385
deposit_rate 0.13175
deposit_rate 0.12400000000000001
deposit_rate 0.1159
d

In [317]:
max_date = df['date'].max() + pd.DateOffset(months=1)
max_date

Timestamp('2024-12-01 00:00:00')

In [318]:
from dateutil.relativedelta import relativedelta
from datetime import datetime

# Обработка первичных вкладов.
# Убираю периоды "будущей доходности".
# max_date = df['date'].max()
correct_income = []
for i, row in df_income.iterrows():
    if max_date < row["end_date"]:
        # n_month = relativedelta(max_date, )
        n_month = (max_date.year - row["start_date"].year) * 12 + (max_date.month - row["start_date"].month)
        row["income"] = n_month / 12 * row["income"]
        row["end_date"] = max_date
        
        correct_income.append(row)
    else:
        correct_income.append(row)
        

In [319]:
max_date, row["end_date"]

(Timestamp('2024-12-01 00:00:00'), Timestamp('2024-12-01 00:00:00'))

In [320]:
df_income_correct = pd.DataFrame(correct_income)

In [321]:
df_income_correct

,start_date,end_date,principal,rate,income
0,2012-01-01,2013-01-01,100000,0.09514,9514.0000
1,2012-02-01,2013-02-01,15000,0.09561,1434.1500
2,2012-03-01,2013-03-01,15000,0.09521,1428.1500
3,2012-04-01,2013-04-01,15000,0.09538,1430.7000
4,2012-05-01,2013-05-01,15000,0.09770,1465.5000
...,...,...,...,...,...
150,2024-07-01,2024-12-01,15000,0.16574,1035.8750
151,2024-08-01,2024-12-01,15000,0.17275,863.7500
152,2024-09-01,2024-12-01,15000,0.17633,661.2375
153,2024-10-01,2024-12-01,15000,0.19780,494.5000


# Расчёт повторных инвестиций.

In [322]:
income_new_all = []
for i, row in df_income_correct.iterrows():
    #row["end_date"]
    income_all_reinvest = []
    end_date_iter = row['end_date']
    total_money = row["principal"] + row["income"]
    while True:
        if end_date_iter < max_date:
            n_month_for_end = (max_date.year - end_date_iter.year) * 12 + (max_date.month - end_date_iter.month)
            n_month_deposit = min([12, n_month_for_end]) # Сколько месяцев вклада произошло.
            rate_in_date = df_rates.loc[df_rates.date == end_date_iter, "rate"].mean()
            new_income = (rate_in_date * total_money) * (n_month_deposit / 12)
            
            total_money += new_income
            end_date_iter = end_date_iter + pd.DateOffset(months=12)
            income_all_reinvest.append(new_income)
        else:
            break    
    
    income_new_all.append(income_all_reinvest)
    # break

In [323]:
df_income_new_all = pd.DataFrame(income_new_all)

In [324]:
i = 2
for col in list(df_income_new_all):
     col_name = "income_" + str(i)
     i += 1
     df_income_correct[col_name] = df_income_new_all[col]

In [325]:
df_income_correct[['principal', 'income', 'income_2', 'income_3']].sum().sum()

3004396.206269422

In [333]:
df_income_correct

,start_date,end_date,principal,rate,income,income_2,income_3,income_4,income_5,income_6,income_7,income_8,income_9,income_10,income_11,income_12,income_13
0,2012-01-01,2013-01-01,100000,0.09514,9514.0000,10568.101000,9978.822593,19931.836541,14997.776086,13859.205042,12957.663754,14441.179524,12224.353606,9800.696005,17663.813811,19994.714519,36061.066063
1,2012-02-01,2013-02-01,15000,0.09561,1434.1500,1616.298653,1492.772104,2873.830612,2187.680043,1981.911115,1851.493786,2131.438317,1677.963951,1440.175073,2628.315419,2878.408464,4843.779239
2,2012-03-01,2013-03-01,15000,0.09521,1428.1500,1638.707963,1507.679297,2711.073410,2181.538428,1968.382145,1760.342025,2164.033273,1563.535187,1445.812676,6844.034078,3086.772024,4821.569988
3,2012-04-01,2013-04-01,15000,0.09538,1430.7000,1633.211580,1556.205983,2584.950489,2142.567016,1898.628583,1679.760874,2104.225948,1629.741693,1467.124037,5492.475866,2964.439887,4111.829080
4,2012-05-01,2013-05-01,15000,0.09770,1465.5000,1595.506950,1559.567950,2432.951288,2151.321480,1916.539838,1663.932384,2090.011762,1645.533267,1555.239475,4000.554827,2831.173698,3480.761187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2024-07-01,2024-12-01,15000,0.16574,1035.8750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,2024-08-01,2024-12-01,15000,0.17275,863.7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,2024-09-01,2024-12-01,15000,0.17633,661.2375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,2024-10-01,2024-12-01,15000,0.19780,494.5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [332]:
print(df_income_correct['principal'].sum(), df_investments['amount'].sum())
df_income_correct

2410000 2410000


,start_date,end_date,principal,rate,income,income_2,income_3,income_4,income_5,income_6,income_7,income_8,income_9,income_10,income_11,income_12,income_13
0,2012-01-01,2013-01-01,100000,0.09514,9514.0000,10568.101000,9978.822593,19931.836541,14997.776086,13859.205042,12957.663754,14441.179524,12224.353606,9800.696005,17663.813811,19994.714519,36061.066063
1,2012-02-01,2013-02-01,15000,0.09561,1434.1500,1616.298653,1492.772104,2873.830612,2187.680043,1981.911115,1851.493786,2131.438317,1677.963951,1440.175073,2628.315419,2878.408464,4843.779239
2,2012-03-01,2013-03-01,15000,0.09521,1428.1500,1638.707963,1507.679297,2711.073410,2181.538428,1968.382145,1760.342025,2164.033273,1563.535187,1445.812676,6844.034078,3086.772024,4821.569988
3,2012-04-01,2013-04-01,15000,0.09538,1430.7000,1633.211580,1556.205983,2584.950489,2142.567016,1898.628583,1679.760874,2104.225948,1629.741693,1467.124037,5492.475866,2964.439887,4111.829080
4,2012-05-01,2013-05-01,15000,0.09770,1465.5000,1595.506950,1559.567950,2432.951288,2151.321480,1916.539838,1663.932384,2090.011762,1645.533267,1555.239475,4000.554827,2831.173698,3480.761187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,2024-07-01,2024-12-01,15000,0.16574,1035.8750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,2024-08-01,2024-12-01,15000,0.17275,863.7500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,2024-09-01,2024-12-01,15000,0.17633,661.2375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,2024-10-01,2024-12-01,15000,0.19780,494.5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [334]:
df_income_correct.to_excel("Итоги прибылей.xlsx")

In [329]:
df_income[["principal", "income"]].sum().sum()

2625371.75

In [ ]:
df_investments['amount'].sum()

In [ ]:
# Ошибки
# Сумма вкладов. 
# Исправить дату окончания. 